In [38]:
!pip install tmdbsimple

In [39]:
import json
with open(r"C:\Users\Antho\.secret\tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [40]:
import tmdbsimple as tmdb
tmdb.API_KEY = login["api-key"]

In [41]:
import os, time
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'genres.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_genres.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'tmdb_api_results_2004.json',
 'tmdb_api_results_2005.json',
 'tmdb_api_results_2006.json',
 'tmdb_api_results_2007.json',
 'tmdb_api_results_2008.json',
 'tmdb_api_results_2009.json',
 'tmdb_api_results_2010.json',
 'tmdb_data.csv.gz']

In [42]:
def write_json(new_data, filename):
    with open(filename, "r+") as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

In [43]:
import pandas as pd
basics = pd.read_csv("Data/title_basics.csv.gz")

In [109]:
YEARS_TO_GET = [2002]
errors = []

In [110]:
from tqdm.notebook import tqdm_notebook
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = "Years", position = 0):
    JSON_FILE = f"{FOLDER}tmdb_api_results_{YEAR}.json"
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE, "w") as f:
            json.dump([{"imdb_id" : 0}], f)

Years:   0%|          | 0/1 [00:00<?, ?it/s]

In [111]:
df = basics.loc[basics["startYear"] == YEAR].copy()
movie_ids = df["tconst"].copy()

In [112]:
previous_df = pd.read_json(JSON_FILE)
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]

In [113]:
def get_movie_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases["countries"]:
        if c["iso_3166_1"] == "US":
            info["certification"] = c["certification"]
    return info

In [114]:
test_movie_ids = ["tt0848228", "tt0332280"]
test_results = []
for id in test_movie_ids:
    test_movie_info = get_movie_rating(id)
    test_results.append(test_movie_info)
pd.DataFrame(test_results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,27719,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,9898,PG-13


In [115]:
for movie_id in tqdm_notebook(movie_ids_to_get, desc = f"Movies from {YEAR}",
                             position = 1, leave = True):
    try:
        temp = get_movie_rating(movie_id)
        write_json(temp, JSON_FILE)
        time.sleep(.02)
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2002:   0%|          | 0/1517 [00:00<?, ?it/s]

In [116]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                    compression = "gzip", index = False)

In [117]:
print(f"- Total errors: {len(errors)}")

- Total errors: 284
